# Provedores de Inferência da Hugging Face

 > Aviso: Este post foi traduzido para o português usando um modelo de tradução automática. Por favor, me avise se encontrar algum erro.

Está claro que o maior hub de modelos de inteligência artificial é a Hugging Face. E agora estão oferecendo a possibilidade de fazer inferência de alguns de seus modelos em provedores de GPUs serverless

Um desses modelos é [Wan-AI/Wan2.1-T2V-14B](https://huggingface.co/Wan-AI/Wan2.1-T2V-14B), que no momento de escrever este post, é o melhor modelo de geração de vídeo open source, como se pode ver na [Artificial Analysis Video Generation Arena Leaderboard](https://artificialanalysis.ai/text-to-video/arena?tab=Leaderboard)
![video generation arena leaderboard](https://images.maximofn.com/Video-arena-leaderboard-wan21.webp)

Se nós olharmos para seu [modelcard](https://huggingface.co/Wan-AI/Wan2.1-T2V-14B), podemos ver à direita um botão que diz `Replicate`.
![Wan2.1-T2V-14B modelcard](https://images.maximofn.com/Wan2.1-T2V-14B.webp)

## Provedores de inferência

Se formos na página de configuração dos [Inference providers](https://huggingface.co/settings/inference-providers) veremos algo assim:
![Provedores de Inferência](https://images.maximofn.com/Inference%20Providers.webp)
Onde podemos clicar no botão com uma chave para inserir a API KEY do provedor que quisermos usar, ou deixar selecionada a opção com dois pontos. Se escolhermos a primeira opção, será o provedor quem nos cobrará pela inferência, enquanto na segunda opção será a Hugging Face quem nos cobrará pela inferência. Então, faça o que for melhor para você.

## Inferência com Replicate

No meu caso, obtive uma API KEY do Replicate e a adicionei a um arquivo chamado `.env`, onde armazenarei as API KEYS e que não deve ser enviado para o GitHub, GitLab ou o repositório do seu projeto.
O `.env` deve ter este formato
``` python
HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS="hf_aL...AY"REPLICATE_API_KEY="r8_Sh...UD"```

Onde `HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS` é um token que você precisa obter a partir do [Hugging Face](https://huggingface.co/settings/tokens) e `REPLICATE_API_KEY` é a API KEY do Replicate, que você pode obter a partir do [Replicate](https://replicate.com/account/api-tokens).

### Leitura das chaves API

A primeira coisa que temos que fazer é ler as chaves API do arquivo `.env`

In [1]:
import os
import dotenv
dotenv.load_dotenv()

REPLICATE_API_KEY = os.getenv("REPLICATE_API_KEY")
HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS = os.getenv("HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS")

### Logging no hub da Hugging Face

Para poder usar o modelo de Wan-AI/Wan2.1-T2V-14B, como está no hub de Hugging Face, precisamos fazer login.

In [2]:
from huggingface_hub import login
login(HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS)

### Cliente de Inferência

Agora criamos um cliente de inferência, temos que especificar o provedor, a API KEY e, neste caso, além disso, vamos estabelecer um tempo de `timeout` de 1000 segundos, porque por padrão é de 60 segundos e o modelo demora bastante para gerar o vídeo.

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(
	provider="replicate",
	api_key=REPLICATE_API_KEY,
	timeout=1000
)

### Geração do vídeo

Já temos tudo para gerar nosso vídeo. Usamos o método `text_to_video` do cliente, passamos o prompt e dizemos qual modelo do hub queremos usar, se não, ele usará o que está por padrão.

In [6]:
video = client.text_to_video(
	"Funky dancer, dancing in a rehearsal room. She wears long hair that moves to the rhythm of her dance.",
	model="Wan-AI/Wan2.1-T2V-14B",
)

### Salvando o vídeo

Por fim, salvamos o vídeo, que é do tipo `bytes`, em um arquivo no nosso disco.

In [7]:
output_path = "output_video.mp4"
with open(output_path, "wb") as f:
    f.write(video)
print(f"Video saved to: {output_path}")

Video saved to: output_video.mp4


## Vídeo gerado

Este é o vídeo gerado pelo modelo
<video src="https://images.maximofn.com/wan2_1_video.webm" controls></video>